'''An implementation of sequence to sequence learning for performing addition
Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)
Input may optionally be reversed, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.
Two digits reversed:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs
Three digits reversed:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs
Four digits reversed:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs
Five digits reversed:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''

All code taken from https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py

In [2]:
from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

/Users/etang/anaconda3/envs/nlu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Character Table

Taking in our string of numbers/operations as a string of characters, we must vectorize each character as a new input to our RNN. 

We first create a table mapping each character in our vocabulary (in our case, this is the numbers 0-9 and addition symbol '+') to a unique integer index. We also create a table mapping back the other way. 

To encode a string, we convert each character in the string to a one-hot vector with the character index as 1 and all other indices as 0. We also feed in the parameter `num_rows` as a maximum length of our string, equivalent to padding all strings to a specified length.

To decode an output array x, we take each vector of the string (representing a component) and decode it back into its most likely character. NOTE: I'm a little unclear about how we deal with variable-length answers. Maybe those are just decoded into space characters?

In [10]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [11]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'


## Model Parameters

Here is where we set the REVERSE parameter as well.

In [12]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

## Data Generation and Preparation

In [13]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


## Setting up Model

In [14]:
# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


## Training, Validating Model

In [15]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 257us/step - loss: 1.8793 - acc: 0.3229 - val_loss: 1.7768 - val_acc: 0.3466
Q 228+103 T 331  ☒ 105 
Q 182+5   T 187  ☒ 25  
Q 98+467  T 565  ☒ 100 
Q 48+915  T 963  ☒ 100 
Q 65+310  T 375  ☒ 500 
Q 103+901 T 1004 ☒ 105 
Q 604+44  T 648  ☒ 100 
Q 23+167  T 190  ☒ 300 
Q 242+530 T 772  ☒ 105 
Q 189+678 T 867  ☒ 101 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s 213us/step - loss: 1.7203 - acc: 0.3620 - val_loss: 1.6472 - val_acc: 0.3886
Q 24+51   T 75   ☒ 21  
Q 182+6   T 188  ☒ 110 
Q 86+982  T 1068 ☒ 1000
Q 490+58  T 548  ☒ 400 
Q 969+760 T 1729 ☒ 1411
Q 465+7   T 472  ☒ 500 
Q 744+61  T 805  ☒ 700 
Q 25+334  T 359  ☒ 348 
Q 776+56  T 832  ☒ 700 
Q 942+56  T 998  ☒ 900 

-----------------

45000/45000 [==============================] - 10s 218us/step - loss: 0.4632 - acc: 0.8445 - val_loss: 0.4041 - val_acc: 0.8708
Q 704+2   T 706  ☑ 706 
Q 62+95   T 157  ☑ 157 
Q 86+77   T 163  ☒ 162 
Q 830+229 T 1059 ☒ 1060
Q 106+470 T 576  ☒ 575 
Q 616+36  T 652  ☑ 652 
Q 40+247  T 287  ☑ 287 
Q 611+550 T 1161 ☒ 1172
Q 6+84    T 90   ☒ 91  
Q 605+361 T 966  ☒ 965 

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s 218us/step - loss: 0.3458 - acc: 0.8969 - val_loss: 0.3251 - val_acc: 0.8984
Q 846+574 T 1420 ☑ 1420
Q 903+359 T 1262 ☒ 1263
Q 140+38  T 178  ☑ 178 
Q 896+73  T 969  ☑ 969 
Q 422+5   T 427  ☑ 427 
Q 626+81  T 707  ☑ 707 
Q 90+972  T 1062 ☑ 1062
Q 9+767   T 776  ☑ 776 
Q 499+61  T 560  ☑ 560 
Q 731+125 T 856  ☑ 856 

--------------------------------------------------
Iteration 17
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 10s 215us/step - loss: 0.0048 - acc: 0.9998 - val_loss: 0.0104 - val_acc: 0.9978
Q 720+344 T 1064 ☑ 1064
Q 771+962 T 1733 ☑ 1733
Q 55+75   T 130  ☑ 130 
Q 76+691  T 767  ☑ 767 
Q 5+766   T 771  ☑ 771 
Q 267+396 T 663  ☑ 663 
Q 340+152 T 492  ☑ 492 
Q 301+268 T 569  ☑ 569 
Q 14+140  T 154  ☑ 154 
Q 551+960 T 1511 ☑ 1511

--------------------------------------------------
Iteration 44
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s 217us/step - loss: 0.0396 - acc: 0.9878 - val_loss: 0.0267 - val_acc: 0.9922
Q 584+871 T 1455 ☑ 1455
Q 438+8   T 446  ☑ 446 
Q 83+815  T 898  ☑ 898 
Q 46+60   T 106  ☑ 106 
Q 39+65   T 104  ☑ 104 
Q 668+67  T 735  ☑ 735 
Q 742+985 T 1727 ☑ 1727
Q 939+97  T 1036 ☑ 1036
Q 24+998  T 1022 ☑ 1022
Q 635+728 T 1363 ☑ 1363

--------------------------------------------------
Iteration 45
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

KeyboardInterrupt: 

In [17]:
history = model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))

Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 19s 425us/step - loss: 0.0456 - acc: 0.9862 - val_loss: 0.0103 - val_acc: 0.9974


In [19]:
history.history['val_loss']

[0.010345782670378684]